### SQLChatMessageHistory
* 채팅메시지를 SQL 데이터베이스 저장하는 클래스

**주요특징**
* SQL 데이터베이스를 사용해 채팅 기록을 영구적으로 저장
* 세션 ID를 통한 대화 구분 및 관리 가능
* SQLAlchemy를 사용해 다양한 데이터베이스 사용 가능
* 메시지 CRUD 가능

* session_id : 사용자 이름, 이메일 같이 고유 식별자
* connection : 데이터베이스 연결을 지정하는 문자

In [19]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id="sql_history",
    connection="sqlite:///sqlite.db"
)

In [20]:
chat_message_history.add_user_message(
    "안녕 만나서 반가워"
)

In [21]:
chat_message_history.add_ai_message(
    "안녕 나도 만나서 반가워"
)

In [22]:
chat_message_history.messages

[HumanMessage(content='안녕 만나서 반가워', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 나도 만나서 반가워', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕 만나서 반가워', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 나도 만나서 반가워', additional_kwargs={}, response_metadata={})]

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        
        MessagesPlaceholder(variable_name="chat_history"),
        
        ("human", "{question}")
    ]
)

chain = prompt | ChatOpenAI(temperature=0, model="gpt-4o") | StrOutputParser()

In [24]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db"
    )

In [25]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_field = [
    ConfigurableFieldSpec(
        id="user_id", # 설정값의 고유 식별자
        annotation=str, # 설정값의 데이터 타입
        name="USER_ID", # 설정값의 이름
        description="Unique identifier for a user", # 설정값에 대한 설명
        default="", # 기본값
        is_shared=True # 여러 대화에서 공유되는 값인지 여부
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="CONVERSATION_ID",
        description="Unique identifier for a conversation",
        default="",
        is_shared=True
    )
]

In [26]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history, # 대화기록을 가져오는 함수
    input_messages_key="question", # 입력 메시지 키
    history_messages_key="chat_history", # 대화 기록 메시지의 키
    history_factory_config = config_field # 대화기록 조회시 참고할 파라미터
)

In [27]:
config = {"configurable" : {"user_id" : "user1", "conversation_id": "conversation1"}}

In [28]:
chain_with_history.invoke({"question": "안녕 반가워 내이름은 bear야"}, config)

'안녕하세요, Bear님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [29]:
config = {"configurable" : {"user_id" : "user1", "conversation_id": "conversation1"}}

chain_with_history.invoke({"question": "내이름이 뭐라고?"}, config)

'당신의 이름은 Bear라고 하셨습니다. 맞나요?'

In [31]:
config = {"configurable" : {"user_id" : "user2", "conversation_id": "conversation1"}}

chain_with_history.invoke({"question": "내이름이 뭐라고?"}, config)

'죄송하지만, 저는 사용자의 이름을 알 수 없습니다. 대신 다른 질문이나 도움이 필요하시면 말씀해 주세요!'